In [1]:
import pandas as pd 
import os
import numpy as np
import utils.utils as utils
import generators.llm_generators as llm_generators
import modeling.payroll as payroll
import modeling.erp as erp
from typing import Dict, List, Tuple, Optional


In [4]:
import json
from typing import Dict, Any

import pandas as pd 
import utils.prompt_utils as prompt_utils
import utils.utils as utils
import generators.random_generators as random_generators
import json
from typing import Dict, Any
import json
import requests
import dotenv
from openai import OpenAI
import os
from pdfminer.high_level import extract_text
import fitz  # PyMuPDF
import re
import json
from typing import Dict, Any

# --- Init OpenAI client ---
def get_openai_client():
    return OpenAI(api_key=API_KEY)

# ---- Load API key from .env ----
dotenv.load_dotenv()
API_KEY = dotenv.get_key('.env', 'api_key')

def _read_text(path: str, max_chars: int = 8_000, enc: str = "utf-8") -> str:
    try:
        with open(path, "r", encoding=enc) as f:
            return f.read()[:max_chars]
    except FileNotFoundError:
        return ""

def generate_context_numbers_llm(
    company_name: str,
    model: str = "gpt-4.1",
    temp: float = 0.7,
) -> Dict[str, Any]:
    """
    Generate synthetic-scale configuration counts and a revenue_downscale factor
    based on the ratio between the real-world company size and the demo size.
    """
    client = get_openai_client()

    # Load company context (e.g. report summary)
    #input_file = f"data/inputdata/reports/generated/{company_name}_context_report.txt"
    ctx = None #_read_text(input_file)

    # ===============================
    # SYSTEM PROMPT
    # ===============================
    system = f"""
    You are a precise data configurator. Return ONLY a valid **minified** JSON object and nothing else.

    Schema:
    {{
        "company_name": str,
        "count_bu": int,
        "count_emp": int,
        "count_parties": int,
        "count_rev": int,
        "count_cogs": int,
        "extra_categories": {{
            "OPEX": int,
            "Inventory": int,
            "EBITA": int, 
            "EBIT": int,
            "BalanceSheet": int,
            "OtherRevenue": int
        }},
        "revenue_downscale": float
    }}

    Guidelines:
    - Treat the company as **large** or **enterprise** based on context.
    - Use the following synthetic ranges for this demo generator:
        "count_bu":       (5,  20),
        "count_emp":      (200, 600),
        "count_parties":  (80, 150),
        "count_rev":      (100, 250),
        "count_cogs":     (80, 250),
        "extra_categories": {{
            "OPEX": (100, 200),
            "Inventory": (100, 200),
            "Fixed cost": (100, 200),
            "EBITDA": (50, 120),
            "EBIT": (40, 100),
            "BalanceSheet": (80, 150),
            "OtherRevenue": (20, 60)
        }}
    - All numbers must be positive integers.
    - Infer approximate **real-world headcount** for the company from context or general knowledge.
      Examples:
        • LEGO ~5000 (Denmark) / ~18000 (global)
        • Novo Nordisk ~20000 (Denmark)
        • Vestas ~29000 (global)
        • Pandora ~3000 (Denmark)
        • Arla ~20000 (global)
    - Compute: revenue_downscale = synthetic_employee_count / real_employee_count. Round to 3 decimal places.
    - Clip revenue_downscale between (0.005, 1.0)
    - If unsure of real employee count, assume 5000 for a large Danish company.
    - Output only the JSON, no prose.

    Context:
    {ctx if ctx else "No context found. Use large Danish company defaults."}
    """.strip()

    # ===============================
    # USER PROMPT
    # ===============================
    user = f"""
    Company: {company_name}
    Task: Produce a configuration dictionary using the schema above.
    Include a realistic revenue_downscale based on the relative size
    of this synthetic setup (a few hundred employees) versus the real company.
    """.strip()

    # ===============================
    # LLM CALL
    # ===============================
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        temperature=temp,
    )

    raw = (resp.choices[0].message.content or "").strip()

    # ===============================
    # PARSE RESPONSE OR FALLBACK
    # ===============================
    try:
        data = json.loads(raw)
    except Exception:
        print("⚠️ LLM output invalid — using fallback defaults.")
        data = {
            "company_name": company_name,
            "count_bu": 15,
            "count_emp": 300,
            "count_parties": 100,
            "count_rev": 150,
            "count_cogs": 100,
            "extra_categories": {"Inventory": 100, "BalanceSheet": 80, "OPEX": 120, "OtherRevenue": 40, "EBITDA": 60, "EBIT": 50},
            "revenue_downscale": 0.06,  # e.g. 300 synthetic / 5000 real
        }

    data["company_name"] = company_name
    return data


In [5]:
import pandas as pd 
import os
import numpy as np
import utils.utils as utils
import generators.llm_generators as llm_generators
import modeling.payroll as payroll
import modeling.erp as erp
from typing import Dict, List, Tuple, Optional

# ------------------------------------------------------------
# Helpers
# ------------------------------------------------------------

def _validate_length(df: pd.DataFrame, expected: int, name: str) -> None:
    if len(df) != expected:
        raise ValueError(f"Expected {expected} rows in {name}, but got {len(df)}.")
    
def _even_chunks(expected_count: int, max_chunk_size: int = 100) -> list[int]:
    """
    Split expected_count into roughly equal chunk sizes, each ≤ max_chunk_size.
    Example:
        _even_chunks(450, 100) -> [100, 100, 100, 100, 50]
    """
    # Number of chunks needed
    n_chunks = int(np.ceil(expected_count / max_chunk_size))

    # Base size for each chunk
    base = expected_count // n_chunks
    rem = expected_count % n_chunks

    # First 'rem' chunks get +1
    chunks = np.full(n_chunks, base, dtype=int)
    chunks[:rem] += 1

    return chunks.tolist()

def _concat_exact(dfs: List[pd.DataFrame], expected: int, name: str) -> pd.DataFrame:
    """
    Concatenate and then trim or sample to hit the exact expected row count.
    Useful when LLM returns +/- a few rows per chunk.
    """
    if not dfs:
        raise ValueError(f"No dataframes produced for {name}.")
    out = pd.concat(dfs, ignore_index=True)

    if len(out) == expected:
        return out

    if len(out) > expected:
        # Downsample deterministically (but allow seed override upstream if needed)
        # Keep stable subset but preserve variety by sampling w/o replacement
        out = out.sample(n=expected, replace=False, random_state=42).reset_index(drop=True)
        return out

    # If fewer rows than requested, upsample with replacement (last resort)
    need = expected - len(out)
    if len(out) == 0:
        raise ValueError(f"LLM produced 0 rows for {name}, cannot upsample.")
    boost = out.sample(n=need, replace=True, random_state=43)
    out = pd.concat([out, boost], ignore_index=True)
    return out


import pandas as pd
import os

def generate_item_id(prefix="ITEM", start_num=1000):
    """Generator for sequential item IDs with a prefix."""
    counter = start_num
    while True:
        yield f"{prefix}{counter:04d}"
        counter += 1


def star_extra_cat(df_extra_cat: pd.DataFrame, prefix: str, id_start: int):
    """
    Build a star-schema subset for a given extra category.
    Returns:
        - fact table (df_fact_extra)
        - dim_party
        - dim_item
    """
    df_extra_cat = df_extra_cat.copy()

    # --- 1. Parties ---
    df_extra_parties = (
        df_extra_cat[["party_id", "party_name"]]
        .drop_duplicates("party_name")
        .dropna()
        .reset_index(drop=True)
    )

    # --- 2. Items ---
    df_extra_items = (
        df_extra_cat[["item_name", "unit_price"]]
        .drop_duplicates("item_name")
        .reset_index(drop=True)
    )

    item_id_gen_extra = generate_item_id("", id_start)
    df_extra_items["item_id"] = [next(item_id_gen_extra) for _ in range(len(df_extra_items))]

    # --- 3. Map items into main table ---
    rev_map = df_extra_items[["item_id", "item_name"]]
    df_extra_cat = df_extra_cat.merge(rev_map, on="item_name", how="left")
    df_extra_cat["item_id"] = df_extra_cat["item_id"].astype("Int64")

    # --- 4. Compute quantity & cleanup ---
    df_extra_cat["quantity"] = (df_extra_cat["annual_spend"] / df_extra_cat["unit_price"]).round().astype("Int64")
    df_extra_cat.rename(columns={"annual_spend": "amount"}, inplace=True)
    df_extra_cat.drop(columns=["item_name", "bu_name", "party_name", "markdown"], inplace=True, errors="ignore")

    df_extra_fact = df_extra_cat[
        ["bu_id", "party_id", "account_id", "item_id", "quantity", "unit_price", "amount", "category"]
    ].reset_index(drop=True)

    return df_extra_fact, df_extra_parties, df_extra_items


def build_and_save_extracategories(extra_categories: dict, output_dir="data/outputdata/extracatagories"):
    """
    Run star_extra_cat() for each extra category in the dictionary.
    Saves outputs into structured CSV files.
    
    Parameters:
        extra_categories: dict where key = category name, value = DataFrame
        output_dir: root output path
    """
    os.makedirs(output_dir, exist_ok=True)

    id_counter = 1000

    for cat_name, df_cat in extra_categories.items():
        print(f"▶ Processing extra category: {cat_name}")

        df_fact, df_parties, df_items = star_extra_cat(df_cat, prefix=cat_name.upper(), id_start=id_counter)
        id_counter += len(df_items) + 1  # increment ID counter so no overlaps

        # Save to output folder
        cat_folder = os.path.join(output_dir, cat_name)
        os.makedirs(cat_folder, exist_ok=True)

        df_fact.to_csv(os.path.join(cat_folder, f"{cat_name}_fact.csv"), index=False)
        df_parties.to_csv(os.path.join(cat_folder, f"{cat_name}_parties.csv"), index=False)
        df_items.to_csv(os.path.join(cat_folder, f"{cat_name}_items.csv"), index=False)

        print(f"   ✓ Saved {cat_name} tables to {cat_folder}")

    print("✅ All extra categories processed successfully.")

# ------------------------------------------------------------
# Main generator
# ------------------------------------------------------------
def generate_items_llm(
    company_name: str,
    financial_totals: Dict[str, float],   # e.g., {"Revenue": 1.2e9, "COGS": -8.0e8}
    *,
    count_bu: int = 15,
    count_emp: int = 300,
    count_parties: int = 100,
    count_rev: int = 150,
    count_cogs: int = 100,
    # Extra categories, e.g. {"OPEX": 200, "OtherRevenue": 40}
    extra_categories: Optional[Dict[str, int]] = None,
    # LLM knobs
    model: str = "gpt-4.1",
    temp: float = 0.7,
    # Chunking: keep chunks modest so prompts stay fast & stable
    max_chunk_size: int = 100,
    save_to_csv: bool = True,
    output_dir: str = "data/outputdata_v1"
) -> Dict[str, pd.DataFrame]:
    """
    New unified generator:
    - Generates business units, roles/departments, parties
    - Generates Revenue, COGS (chunked), plus any extra categories (chunked)
    - Returns dict of DataFrames and optionally writes CSVs
    - Enforces exact requested row counts for each generated table
    """

    # ----------------------------
    # 1) Top-level dimensions
    # ----------------------------
    print("Generating top-level dimensions...")
    df_coa = pd.read_csv("data/inputdata/coa_general.csv", sep = ";")
    df_accounts = llm_generators.tailor_coa_names_llm(df_coa, company_name="Lego")

    df_bus = llm_generators.generate_bus_llm(company_name=company_name, model=model, temp=temp, count=count_bu)
    _validate_length(df_bus, count_bu, "business_units")

    df_roles, df_departments = llm_generators.generate_roles_and_departments_llm(
        company_name=company_name,
        count=count_emp,
        df_business_units=df_bus
    )
    _validate_length(df_roles, count_emp, "roles")
    _validate_length(df_departments, df_departments.shape[0], "departments")

    df_parties = llm_generators.generate_parties(
        company_name=company_name, model=model, temp=temp, count=count_parties, df_business_units=df_bus
    )
    _validate_length(df_parties, count_parties, "parties")
    print("Top-level dimensions generated.")

    # Optional: mean pay + payroll (parity with your old function)
    # If you don’t need payroll here, you can remove this block.
    print("Generating payroll data...")
    mean_pay = llm_generators.estimate_mean_pay_llm(company_name)
    df_employees = utils.sample_employees(count_emp, if_fullname=False)

    df_pay  = payroll.create_pay_roll(df_roles=df_roles, df_employees=df_employees, mean_pay=mean_pay, if_long=False)
    df_payroll = payroll.create_pay_roll(df_roles=df_roles, df_employees=df_employees, mean_pay=mean_pay, if_long=True)
    #df_pay, df_payroll, df_payline = create_payroll_data(
    #    df_pay=df_pay, df_payroll=df_payl, df_department=df_departments
    #)
    print("Payroll data generated.")
    # ----------------------------
    # 2) Line-item categories
    # ----------------------------
    outputs: Dict[str, pd.DataFrame] = {
        "accounts": df_accounts,
        "business_units": df_bus,
        "roles": df_roles,
        "departments": df_departments,
        "parties": df_parties,
        "pay": df_pay,
        "payroll": df_payroll,
        #"payline": df_payline,
    }

    print("Generating line-item categories...")
    def _generate_category(
        category_name: str, expected_count: int, total_amount: float
    ) -> pd.DataFrame:
        """
        Chunk, call LLM per chunk, then concat & enforce exact rows.
        Financial total is split proportionally to chunk size, so the
        sum of chunk totals equals the category total.
        """
        if expected_count <= 0:
            return pd.DataFrame()

        chunk_sizes = _even_chunks(expected_count, max_chunk_size)
        dfs = []
        remaining_total = float(total_amount)
        remaining_rows = int(expected_count)

        for i, chunk_n in enumerate(chunk_sizes):
            # Split the total amount proportionally
            if i < len(chunk_sizes) - 1 and remaining_rows > 0:
                frac = chunk_n / remaining_rows
                chunk_total = remaining_total * frac
            else:
                chunk_total = remaining_total

            if i == 1: 
                print("Generating ", expected_count, " for", category_name, "!")
            
            df_chunk = llm_generators.generate_line_items_llm(
                company_name=company_name,
                count=chunk_n,
                category_name=category_name,
                financial_total=chunk_total,
                df_business_units=df_bus,
                df_parties=df_parties,
                df_accounts=df_accounts
            )
            dfs.append(df_chunk)

            remaining_total -= chunk_total
            remaining_rows  -= chunk_n

        df_cat = _concat_exact(dfs, expected_count, name=category_name)
        return df_cat

    # Revenue
    if "Revenue" in financial_totals and count_rev > 0:
        df_revenue = _generate_category(
            "Revenue", count_rev, float(financial_totals["Revenue"])
        )
        outputs["revenue"] = df_revenue

    # COGS
    if "COGS" in financial_totals and count_cogs > 0:
        df_cogs = _generate_category(
            "COGS", count_cogs, float(financial_totals["COGS"])
        )
        outputs["cogs"] = df_cogs

    # Any extra categories (e.g. OPEX, OtherIncome, etc.)
    if extra_categories:
        for cat_name, cat_count in extra_categories.items():
            if cat_count <= 0:
                continue
            total = float(financial_totals.get(cat_name, 0.0))
            print("Generating ", cat_count, " for", cat_name, "!")
            df_cat = _generate_category(cat_name, cat_count, total)
            outputs[cat_name.lower()] = df_cat
    
    print("Line-item categories generated.")

    df_revenue, df_cogs = utils.mirror_intercompany_flows(df_revenue, df_cogs)
    df_customers = utils.get_party_list(df_revenue, df_parties, df_bus, "customer")
    df_vendors = utils.get_party_list(df_cogs, df_parties, df_bus, "vendor")

    # Function to generate item IDs
    def generate_item_id(prefix="ITEM", start_num=1000):
        """Generate sequential item IDs with a prefix"""
        counter = start_num
        while True:
            yield f"{prefix}{counter:04d}"
            counter += 1
    print("Processing products, items, and departments...")
    # Create item IDs for products and COGS

    # Any extra categories (e.g. OPEX, OtherIncome, etc.)
    if extra_categories:
        for cat_name, cat_count in extra_categories.items():
            if cat_count <= 0:
                continue
            total = float(financial_totals.get(cat_name, 0.0))
            print("Generating ", cat_count, " for", cat_name, "!")
            df_cat = _generate_category(cat_name, cat_count, total)
            outputs[cat_name.lower()] = df_cat
        
    df_products = df_revenue[["item_name", "unit_price"]].drop_duplicates("item_name")
    df_items = df_cogs[["item_name", "unit_price"]].drop_duplicates("item_name")

    # Generate item IDs for products
    item_id_gen = generate_item_id("", 1000)
    df_products["item_id"] = [next(item_id_gen) for _ in range(len(df_products))]

    # Generate item IDs for COGS items  
    item_id_gen_cogs = generate_item_id("", 2000)
    df_items["item_id"] = [next(item_id_gen_cogs) for _ in range(len(df_items))]

    if save_to_csv:
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(f"{output_dir}/dimensions", exist_ok=True)
        os.makedirs(f"{output_dir}/fact", exist_ok=True)

        # Dimensions
        outputs["accounts"].to_csv(f"{output_dir}/dimensions/accounts.csv", index=False)
        outputs["business_units"].to_csv(f"{output_dir}/dimensions/business_unit.csv", index=False)
        outputs["roles"].to_csv(f"{output_dir}/dimensions/role.csv", index=False)
        outputs["departments"].to_csv(f"{output_dir}/dimensions/department.csv", index=False)
        outputs["parties"].to_csv(f"{output_dir}/dimensions/parties.csv", index=False)
        df_customers.to_csv(f"{output_dir}/dimensions/customer.csv", index=False)
        df_vendors.to_csv(f"{output_dir}/dimensions/vendors.csv", index=False)
        df_items.to_csv(f"{output_dir}/dimensions/cogs_items.csv", index=False)
        df_products.to_csv(f"{output_dir}/dimensions/products.csv", index=False)
        df_revenue.to_csv(f"{output_dir}/dimensions/basis_revenue.csv", index=False)
        df_cogs.to_csv(f"{output_dir}/dimensions/basis_cogs.csv", index=False)
        # Facts
        outputs["payroll"].to_csv(f"{output_dir}/fact/erp_payroll.csv", index=False)
        # Extra categories go to /fact/<cat>.csv
        for k, v in outputs.items():
            if k not in {"business_units","roles","departments","parties","pay","payroll","payline","revenue","cogs"}:
                if isinstance(v, pd.DataFrame) and not v.empty:
                    v.to_csv(f"{output_dir}/fact/{k}_lines.csv", index=False)

    print(f"Full generation for {company_name} completed!")

    outputs: Dict[str, pd.DataFrame] = {
        "accounts": df_accounts,
        "business_units": df_bus,
        "roles": df_roles,
        "departments": df_departments,
        "parties": df_parties,
        "pay": df_pay,
        "payroll": df_payroll,
        "revenue": df_revenue,
        "cogs": df_cogs,
        "customers": df_customers,
        "vendors": df_vendors,
        "products": df_products,
        "cogs_items": df_items,
    }

    return outputs


In [4]:
{'company_name': 'Lego',
 'count_bu': 13,
 'count_emp': 472,
 'count_parties': 127,
 'count_rev': 212,
 'count_cogs': 206,
 'extra_categories': {'OPEX': 186,
  'Inventory': 116,
  'EBITA': 95,
  'EBIT': 87,
  'BalanceSheet': 116,
  'OtherRevenue': 34},
 'revenue_downscale': 0.026}

{'company_name': 'Lego',
 'count_bu': 13,
 'count_emp': 472,
 'count_parties': 127,
 'count_rev': 212,
 'count_cogs': 206,
 'extra_categories': {'OPEX': 186,
  'Inventory': 116,
  'EBITA': 95,
  'EBIT': 87,
  'BalanceSheet': 116,
  'OtherRevenue': 34},
 'revenue_downscale': 0.026}

In [ ]:
def create_data_for_comp(company_name: str, intput_file: file):
    financial_totals = KUBE.get_total_financials(input_file)

    context = generate_context_numbers_llm("Lego")
    downscale = context["revenue_downscale"]

    # Downscale totals for demo purpose
    financial_totals_downscaled = {k: v * downscale for k, v in financial_totals.items()}

    extra_categories_input = ["Inventory"]

    extra_categories_dict = {
        cat: context["extra_categories"][cat]
        for cat in extra_categories_input
        if cat in context["extra_categories"]
    }

    data = generate_items_llm(
        context["company_name"],
        financial_totals_downscaled,
        count_bu=5,#context["count_bu"],
        count_emp=5,#context["count_emp"],
        count_parties=5,#context["count_parties"],
        count_rev=5,#context["count_rev"],
        count_cogs=5,#context["count_cogs"],
        extra_categories=extra_categories_dict,
        save_to_csv=False
    )

    erp_data = create_GL(
        data_generated=data,
        schema_name="uniform_stable",
        years=(2023, 2025),
        save_to_csv=True,
        output_dir="data/outputdata_v1"
    )

    # Using the General Ledger for the final dims: 
    print("Generating Data for report 2:")
    commenting_reporting     = generate_commenting_reporting(erp_data)
    
    print("Generating Invoice Data:")
    invoice_data    = generate_invoice(erp_data)
    
    print("Generating Drivers for reporting:")
    reporting_tools = generate_reporting(erp_data)

    return erp_data

In [7]:
financial_totals = {
    'Revenue':   74325000192.0,
    'COGS':      23569999872.0,
    'Inventory': 6051999744.0,
    'Fixed cost':23438000128.0,
    'EBIT':      18741000192.0
}

context = generate_context_numbers_llm("Lego")
downscale = context["revenue_downscale"]

# Downscale totals for demo purpose
financial_totals_downscaled = {k: v * downscale for k, v in financial_totals.items()}

extra_categories_input = ["Inventory", "Fixed Cost", "EBIT"]

extra_categories_dict = {
    cat: context["extra_categories"][cat]
    for cat in extra_categories_input
    if cat in context["extra_categories"]
}

data = generate_items_llm(
    context["company_name"],
    financial_totals_downscaled,
    count_bu=context["count_bu"],
    count_emp=context["count_emp"],
    count_parties=context["count_parties"],
    count_rev=context["count_rev"],
    count_cogs=context["count_cogs"],
    extra_categories=extra_categories_dict,
    save_to_csv=True
)

Generating top-level dimensions...
Top-level dimensions generated.
Generating payroll data...
Top-level dimensions generated.
Generating payroll data...
Payroll data generated.
Generating line-item categories...
Payroll data generated.
Generating line-item categories...
Generating  213  for Revenue !
Generating  213  for Revenue !
Generating  207  for COGS !
Generating  207  for COGS !
Generating  169  for Inventory !
Generating  169  for Inventory !
Generating  169  for Inventory !
Generating  169  for Inventory !
Generating  81  for EBIT !
Generating  81  for EBIT !
Line-item categories generated.
Processing products, items, and departments...
Generating  169  for Inventory !
Line-item categories generated.
Processing products, items, and departments...
Generating  169  for Inventory !
Generating  169  for Inventory !
Generating  169  for Inventory !
Generating  81  for EBIT !
Generating  81  for EBIT !
Full generation for Lego completed!
Full generation for Lego completed!


In [10]:
data["revenue"]

,bu_id,bu_name,party_id,party_name,account_id,item_name,unit_price,markdown,category,annual_spend
0,BU005,Lego Retail Nordics,CUS008,Tesco,4001,LEGO City Fire Station Set,349.95,0.23,Revenue,16867200.0
1,BU005,Lego Retail Nordics,CUS004,Amazon,4001,LEGO Star Wars Millennium Falcon,1299.00,0.19,Revenue,16867200.0
2,BU005,Lego Retail Nordics,CUS012,"Toys ""R"" Us",4002,Retail promotion bundle discount,-150.00,0.00,Revenue,5728500.0
3,BU011,Lego North America Sales,CUS003,Target,4004,LEGO Friends Heartlake City School,549.00,0.21,Revenue,15594200.0
4,BU011,Lego North America Sales,CUS036,Macy's,4004,LEGO Technic Bugatti Chiron,2999.00,0.18,Revenue,15594200.0
...,...,...,...,...,...,...,...,...,...,...
223,BU010,Lego Americas Distribution Center,BU001,Lego HQ,4007,Set packaging,26400.00,0.20,Revenue,652700.0
224,BU009,Lego Monterrey Factory,BU003,Lego Shared Services Center,4007,Intercompany assembled sets,24100.00,0.34,Revenue,461500.0
225,BU014,Lego Shanghai Factory,BU011,Lego North America Sales,4007,Intercompany minifigure parts,22500.00,0.32,Revenue,410300.0
226,BU016,Lego China Retail,BU011,Lego North America Sales,4007,Intercompany licensed sets,23700.00,0.33,Revenue,359000.0


In [9]:
data["Inventory"]

KeyError: 'Inventory'

In [8]:
import json
from typing import Dict, Any

import pandas as pd 
import utils.prompt_utils as prompt_utils
import utils.utils as utils
import generators.random_generators as random_generators
import json
from typing import Dict, Any
import json
import requests
import dotenv
from openai import OpenAI
import os
from pdfminer.high_level import extract_text
import fitz  # PyMuPDF
import re
import json
from typing import Dict, Any

# --- Init OpenAI client ---
def get_openai_client():
    return OpenAI(api_key=API_KEY)

# ---- Load API key from .env ----
dotenv.load_dotenv()
API_KEY = dotenv.get_key('.env', 'api_key')

def _read_text(path: str, max_chars: int = 8_000, enc: str = "utf-8") -> str:
    try:
        with open(path, "r", encoding=enc) as f:
            return f.read()[:max_chars]
    except FileNotFoundError:
        return ""

def generate_context_numbers_llm(
    company_name: str,
    model: str = "gpt-4.1",
    temp: float = 0.7,
) -> Dict[str, Any]:
    """
    Generate synthetic-scale configuration counts and a revenue_downscale factor
    based on the ratio between the real-world company size and the demo size.
    """
    client = get_openai_client()

    # Load company context (e.g. report summary)
    #input_file = f"data/inputdata/reports/generated/{company_name}_context_report.txt"
    ctx = None #_read_text(input_file)

    # ===============================
    # SYSTEM PROMPT
    # ===============================
    system = f"""
    You are a precise data configurator. Return ONLY a valid **minified** JSON object and nothing else.

    Schema:
    {{
        "company_name": str,
        "count_bu": int,
        "count_emp": int,
        "count_parties": int,
        "count_rev": int,
        "count_cogs": int,
        "extra_categories": {{
            "OPEX": int,
            "Inventory": int,
            "EBITA": int, 
            "EBIT": int,
            "BalanceSheet": int,
            "OtherRevenue": int
        }},
        "revenue_downscale": float
    }}

    Guidelines:
    - Treat the company as **large** or **enterprise** based on context.
    - Use the following synthetic ranges for this demo generator:
        "count_bu":       (5,  20),
        "count_emp":      (200, 600),
        "count_parties":  (80, 150),
        "count_rev":      (100, 250),
        "count_cogs":     (80, 250),
        "extra_categories": {{
            "OPEX": (100, 200),
            "Inventory": (100, 200),
            "EBITDA": (50, 120),
            "EBIT": (40, 100),
            "BalanceSheet": (80, 150),
            "OtherRevenue": (20, 60)
        }}
    - All numbers must be positive integers.
    - Infer approximate **real-world headcount** for the company from context or general knowledge.
      Examples:
        • LEGO ~5000 (Denmark) / ~18000 (global)
        • Novo Nordisk ~20000 (Denmark)
        • Vestas ~29000 (global)
        • Pandora ~3000 (Denmark)
        • Arla ~20000 (global)
    - Compute: revenue_downscale = synthetic_employee_count / real_employee_count. Round to 3 decimal places.
    - Clip revenue_downscale between (0.005, 1.0)
    - If unsure of real employee count, assume 5000 for a large Danish company.
    - Output only the JSON, no prose.

    Context:
    {ctx if ctx else "No context found. Use large Danish company defaults."}
    """.strip()

    # ===============================
    # USER PROMPT
    # ===============================
    user = f"""
    Company: {company_name}
    Task: Produce a configuration dictionary using the schema above.
    Include a realistic revenue_downscale based on the relative size
    of this synthetic setup (a few hundred employees) versus the real company.
    """.strip()

    # ===============================
    # LLM CALL
    # ===============================
    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        temperature=temp,
    )

    raw = (resp.choices[0].message.content or "").strip()

    # ===============================
    # PARSE RESPONSE OR FALLBACK
    # ===============================
    try:
        data = json.loads(raw)
    except Exception:
        print("⚠️ LLM output invalid — using fallback defaults.")
        data = {
            "company_name": company_name,
            "count_bu": 15,
            "count_emp": 300,
            "count_parties": 100,
            "count_rev": 150,
            "count_cogs": 100,
            "extra_categories": {"Inventory": 100, "BalanceSheet": 80, "OPEX": 120, "OtherRevenue": 40, "EBITDA": 60, "EBIT": 50},
            "revenue_downscale": 0.06,  # e.g. 300 synthetic / 5000 real
        }

    data["company_name"] = company_name
    return data


In [21]:
generate_context_numbers_llm("Lego")

{'company_name': 'Lego',
 'count_bu': 13,
 'count_emp': 472,
 'count_parties': 127,
 'count_rev': 212,
 'count_cogs': 206,
 'extra_categories': {'OPEX': 186,
  'Inventory': 116,
  'EBITA': 95,
  'EBIT': 87,
  'BalanceSheet': 116,
  'OtherRevenue': 34},
 'revenue_downscale': 0.026}

In [111]:
# Load data from saved CSV files
output_dir = "data/cbs_new_data"
# Load dimensions
df_accounts.to_csv(f"{output_dir}/dim/CoA.csv")
df_bus.to_csv(f"{output_dir}/dim/business_unit.csv")
df_products.to_csv(f"{output_dir}/dim/products.csv")
df_items.to_csv(f"{output_dir}/dim/cogs_items.csv")

In [12]:
# Load data from saved CSV files
output_dir = "data/outputdata_v1"

# Load dimensions
df_accounts = pd.read_csv(f"{output_dir}/dimensions/accounts.csv")
df_bus = pd.read_csv(f"{output_dir}/dimensions/business_unit.csv")
df_roles = pd.read_csv(f"{output_dir}/dimensions/role.csv")
df_departments = pd.read_csv(f"{output_dir}/dimensions/department.csv")
df_customers = pd.read_csv(f"{output_dir}/dimensions/customer.csv")
df_vendors = pd.read_csv(f"{output_dir}/dimensions/vendors.csv")
df_products = pd.read_csv(f"{output_dir}/dimensions/products.csv")
df_items = pd.read_csv(f"{output_dir}/dimensions/cogs_items.csv")
df_revenue = pd.read_csv(f"{output_dir}/dimensions/basis_revenue.csv")
df_cogs = pd.read_csv(f"{output_dir}/dimensions/basis_cogs.csv")
#df_inventory = pd.read_csv(f"{output_dir}/extracatagories/inventory/inventory_fact.csv")
#df_inventory_parties = pd.read_csv(f"{output_dir}/extracatagories/inventory/inventory_parties.csv")
#df_inventory_items = pd.read_csv(f"{output_dir}/extracatagories/inventory/inventory_items.csv")
# Load facts
#df_payroll = pd.read_csv(f"{output_dir}/fact/erp_payroll.csv")

# Create data dictionary
data = {
    "accounts": df_accounts,
    "business_units": df_bus,
    "roles": df_roles,
    "departments": df_departments,
    "parties": df_customers,  # Using customers as parties
    "pay": None,  # Not saved separately
 #   "payroll": df_payroll,
    "revenue": df_revenue,
    "cogs": df_cogs,
    "customers": df_customers,
    "vendors": df_vendors,
    "products": df_products,
    "cogs_items": df_items,
}

print(f"✓ Loaded data from {output_dir}")
print(f"  - Revenue items: {len(df_revenue):,}")
print(f"  - COGS items: {len(df_cogs):,}")
print(f"  - Business units: {len(df_bus):,}")
print(f"  - Products: {len(df_products):,}")
print(f"  - COGS items: {len(df_items):,}")

✓ Loaded data from data/outputdata_v1
  - Revenue items: 228
  - COGS items: 220
  - Business units: 12
  - Products: 178
  - COGS items: 186


In [13]:
import modeling.erp as erp
import schemas.erp_schemas as erp_schemas

def map_clean(df_lines: pd.DataFrame, df_items: pd.DataFrame) -> pd.DataFrame:
    # 1) Map product names in df_revenue to product_id
    rev_map = df_items[["item_id","item_name"]]
    df_lines = df_lines.merge(rev_map, left_on="item_name", right_on="item_name", how="left")
    df_lines["item_id"] = df_lines["item_id"].astype("Int64")
    df_lines = df_lines.drop(columns=["item_name","item_name", "bu_name","party_name"], inplace=False)
    return df_lines

def create_GL(
    data_generated: Dict[str, pd.DataFrame],
    schema_name: str = "recovery_growth",
    years: tuple = (2023, 2025),
    save_to_csv: bool = True,
    output_dir: str = "data/outputdata_v1"
) -> Dict[str, pd.DataFrame]:
    """
    Create GL basis (actuals + budget) using a predefined schema.
    
    Parameters
    ----------
    data_generated : dict
        Output from generate_items_llm with 'revenue' and 'cogs' DataFrames
    schema_name : str
        Schema to use: 'recovery_growth', 'steady_seasonal', 'declining_business',
        'uniform_stable', or 'volatile_cyclical'
    years : tuple
        (start_year, end_year) for data generation
    save_to_csv : bool
        Whether to save outputs to CSV
    output_dir : str
        Directory for output files
        
    Returns
    -------
    dict
        Dictionary with 'actuals', 'budget', and 'combined' DataFrames
    """
    
    # Load schema
    schema = erp_schemas.get_schema(schema_name)
    print(f"📊 Using Schema: {schema['name']}")
    print(f"   {schema['description']}\n")
        
    df_revenue = map_clean(data_generated["revenue"], data_generated["products"])
    df_cogs = map_clean(data_generated["cogs"], data_generated["cogs_items"])

    print("Annual Spend of sum revenue :" , sum(df_revenue["annual_spend"]))
    print("Annual Spend of sum cogs :" , sum(df_cogs["annual_spend"]))

    # Generate ERP actuals with growth and seasonality
    print("Generating ERP Actuals...")
    erp_revenue = erp.create_erp_data_with_growth(
        df_revenue,
        year=years,
        year_growth_rates=schema['year_growth_rates'],
        monthly_weights=schema['monthly_weights'],
        max_lines_per_doc=25
    )
    
    erp_cogs = erp.create_erp_data_with_growth(
        df_cogs,
        year=years,
        year_growth_rates=schema['year_growth_rates'],
        monthly_weights=schema['monthly_weights'],
        max_lines_per_doc=25
    )
    
    # Combine actuals
    erp_all = pd.concat([erp_revenue, erp_cogs], ignore_index=True)
    erp_all = erp_all.sort_values("date").reset_index(drop=True)
    erp_all['scenario'] = 'Actual'
    
    print(f"✓ Generated {len(erp_all):,} actual transactions")
    
    # Generate budget using schema parameters
    budget_years = list(range(years[0] + 1, years[1] + 1))
    print(f"\nGenerating Budget for {budget_years}...")
    
    budget_all = erp.create_budget_from_erp(
        erp_all,
        budget_years=budget_years,
        **schema['budget_params']
    )
    
    print(f"✓ Generated {len(budget_all):,} budget lines")
    
    # Combine everything
    combined = pd.concat([erp_all, budget_all], ignore_index=True)
    combined = combined.sort_values(['date', 'scenario', 'document_number']).reset_index(drop=True)
    
    # Summary statistics
    print(f"\n{'='*70}")
    print("SUMMARY BY YEAR AND SCENARIO:")
    print(f"{'='*70}")
    summary = combined.groupby([combined['date'].dt.year, 'category', 'scenario'])['amount'].sum().unstack(['category', 'scenario'], fill_value=0)
    print(summary)
    
    # Save to CSV if requested
    if save_to_csv:
        os.makedirs(output_dir, exist_ok=True)
        os.makedirs(f"{output_dir}/fact", exist_ok=True)
        
        erp_all.to_csv(f"{output_dir}/fact/general_ledger_actual.csv", index=False)
        budget_all.to_csv(f"{output_dir}/fact/general_ledger_budget.csv", index=False)
        combined.to_csv(f"{output_dir}/fact/general_ledger_combined.csv", index=False)
        
        print(f"\n✓ Files saved to {output_dir}/fact/")
    
    return {
        "actuals": erp_all,
        "budget": budget_all,
        "combined": combined
    }

In [14]:
def map_clean(df_lines: pd.DataFrame, df_items: pd.DataFrame) -> pd.DataFrame:
    # 1) Map product names in df_revenue to product_id
    rev_map = df_items[["item_id","item_name"]]
    df_lines = df_lines.merge(rev_map, left_on="item_name", right_on="item_name", how="left")
    df_lines["item_id"] = df_lines["item_id"].astype("Int64")
    df_lines = df_lines.drop(columns=["item_name","item_name", "bu_name","party_name"], inplace=False)
    return df_lines

df_revenue = map_clean(data["revenue"], data["products"])

In [ ]:
erp_data = create_GL(
    data_generated=data,
    schema_name="uniform_stable",
    years=(2023, 2025),
    save_to_csv=True,
    output_dir="data/outputdata_v1"
)

📊 Using Schema: Uniform & Stable
   Minimal seasonality, steady predictable growth (2024 baseline)

Annual Spend of sum revenue : 1797667500.0
Annual Spend of sum cogs : 655183600.0
Generating ERP Actuals...
✓ Generated 2,066,040 actual transactions

Generating Budget for [2024, 2025]...
✓ Generated 2,066,040 actual transactions

Generating Budget for [2024, 2025]...
✓ Generated 10,704 budget lines
✓ Generated 10,704 budget lines

SUMMARY BY YEAR AND SCENARIO:

SUMMARY BY YEAR AND SCENARIO:
category          COGS       Revenue         COGS       Revenue
scenario        Actual        Actual       Budget        Budget
date                                                           
2023     -6.438344e+08  1.743213e+09          0.0  0.000000e+00
2024     -6.653662e+08  1.797195e+09  683956600.0  1.848651e+09
2025     -6.901098e+08  1.869660e+09  705091100.0  1.908180e+09
category          COGS       Revenue         COGS       Revenue
scenario        Actual        Actual       Budget       

In [32]:
erp_data["actuals"]

,document_number,date,bu_id,party_id,account_id,item_id,debit_credit,amount,markdown,quantity,customer_id,vendor_id,category,scenario
0,700-58140500,2023-01-02,BU006,CUS030,4002,1129,Credit,187.00,0.00,187,CUS030,None,Revenue,Actual
1,700-10091800,2023-01-02,BU006,CUS030,4004,1020,Credit,2580.00,0.15,20,CUS030,None,Revenue,Actual
2,700-15315100,2023-01-02,BU013,CUS033,4001,1032,Credit,4980.00,0.10,20,CUS033,None,Revenue,Actual
3,700-10091800,2023-01-02,BU006,CUS030,4004,1020,Credit,2580.00,0.15,20,CUS030,None,Revenue,Actual
4,700-15315700,2023-01-02,BU013,CUS033,4001,1032,Credit,4980.00,0.10,20,CUS033,None,Revenue,Actual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066035,700-69114900,2025-12-31,BU006,CUS004,4004,1154,Credit,2697.00,0.22,3,CUS004,None,Revenue,Actual
2066036,700-1532100,2025-12-31,BU006,CUS003,4002,1001,Credit,1749.75,0.20,5,CUS003,None,Revenue,Actual
2066037,700-63573400,2025-12-31,BU009,CUS020,4004,1144,Credit,14995.00,0.22,5,CUS020,None,Revenue,Actual
2066038,700-62992100,2025-12-31,BU015,CUS003,4001,1140,Credit,2995.00,0.15,5,CUS003,None,Revenue,Actual


In [33]:
df_products.head(30)

,item_name,unit_price,item_id
0,LEGO City Fire Brigade Set,349.95,1000
1,Promotional Allowances,349.95,1001
2,LEGO Friends Heartlake City Hotel,399.95,1002
3,LEGO Star Wars Millennium Falcon,749.95,1003
4,Online Discount Promotions,749.95,1004
5,LEGO Technic Lamborghini Sián FKP 37,2999.00,1005
6,LEGO Classic Bricks and Animals,149.95,1006
7,Seasonal Sales Allowances,149.95,1007
8,LEGO Ninjago Jungle Dragon,479.00,1008
9,Trade Promotion Deductions,479.00,1009


In [34]:
data["revenue"]

,bu_id,bu_name,party_id,party_name,account_id,item_name,unit_price,markdown,category,annual_spend
0,BU006,Lego North America Sales,CUS003,Target,4004,LEGO City Fire Brigade Set,349.95,0.14,Revenue,18414200.0
1,BU006,Lego North America Sales,CUS003,Target,4002,Promotional Allowances,349.95,0.20,Revenue,7128100.0
2,BU006,Lego North America Sales,CUS003,Target,4001,LEGO Friends Heartlake City Hotel,399.95,0.10,Revenue,15444200.0
3,BU006,Lego North America Sales,CUS004,Amazon,4004,LEGO Star Wars Millennium Falcon,749.95,0.13,Revenue,16632200.0
4,BU006,Lego North America Sales,CUS004,Amazon,4002,Online Discount Promotions,749.95,0.18,Revenue,5940100.0
...,...,...,...,...,...,...,...,...,...,...
223,BU015,Lego Discovery Center Chicago,BU007,Lego USA Distribution Center,4007,Intercompany transfer of retail sets,19500.00,0.19,Revenue,352100.0
224,BU003,Lego Shared Services Center,BU002,Lego Billund Factory,4007,Shared service allocation,47000.00,0.20,Revenue,1631500.0
225,BU001,Lego HQ,BU005,Lego Kladno Factory,4007,IT infrastructure allocation,41000.00,0.21,Revenue,1087700.0
226,BU001,Lego HQ,BU008,Lego Monterrey Factory,4007,Finance process allocation,38500.00,0.22,Revenue,699200.0


In [35]:
#df_products = data["products"]
#df_products.sort_values("unit_price").head(30)

In [36]:
actuals = erp_data["actuals"]

In [37]:
df_accounts[df_accounts["AccountKey"] == 4005]

,AccountKey,name,AccountType,AcSign,BudgetArea,GLLevel01,GLLevel02,GLLevel03,GLLevel04,GLLevel05,CFLevel01,CFLevel02,CFLevel03,CFLevel04,CFLevel05,CFLevel06,CFLevel07,CFLevel08
38,4005,Wholesale Sales Allowances,P,-1,Margin,Net Income,Net Sales,Discounts,Trade,NaN,CashFlow,Cash flows From Operating Activities,Cash Generated From Operations,Profit Before Taxation,NaN,NaN,NaN,NaN


In [ ]:
mask = actuals["category"] == "Revenue"
actuals.loc[mask, "amount_counter"] = (
    actuals.loc[mask, "amount"] * -1 * (1 - actuals.loc[mask, "markdown"])
)


In [41]:

mask_cogs = actuals["category"] == "COGS"
actuals.loc[mask_cogs, "amount_counter"] = (
    actuals.loc[mask_cogs, "amount"] * -1 * (1 + actuals.loc[mask_cogs, "markdown"])
)

In [43]:
actuals.loc[actuals["markdown"] == 0, "amount_counter"] = 0

In [44]:
actuals.drop(columns=["markdown"], inplace=True)

In [46]:
actuals[actuals["scenario"] == "budget"]

,document_number,date,bu_id,party_id,account_id,item_id,debit_credit,amount,quantity,customer_id,vendor_id,category,scenario,amount_counter


In [47]:
actuals.drop(columns=["category"], inplace=True)

In [48]:
budget = erp_data["budget"]

In [57]:
actuals_cols = actuals.columns.tolist()

In [58]:
budget=budget[actuals_cols]

In [59]:
combined = pd.concat([actuals, budget], ignore_index=True, axis=0)

In [62]:
combined.to_csv("data/cbs_new_data/fact/general_ledger.csv", index=False)

In [52]:
budget["amount_counter"] = 0

In [49]:
budget.drop(columns=["category", "markdown"], inplace=True)
budget["amount_counter"] = 0

KeyError: "['markdown'] not found in axis"

In [15]:
actuals[actuals["item_id"]==1128]

,document_number,date,bu_id,party_id,account_id,item_id,debit_credit,amount,markdown,quantity,customer_id,vendor_id,category,scenario
1013,DOC-031697,2023-01-10,BU013,CUS063,4004,1128,Credit,3168.0,0.29,32.0,CUS063,None,Revenue,Actual
3555,DOC-031698,2023-01-30,BU013,CUS063,4004,1128,Credit,3564.0,0.29,36.0,CUS063,None,Revenue,Actual
4104,DOC-031700,2023-02-02,BU013,CUS063,4004,1128,Credit,1089.0,0.29,11.0,CUS063,None,Revenue,Actual
4161,DOC-031700,2023-02-02,BU013,CUS063,4004,1128,Credit,1188.0,0.29,12.0,CUS063,None,Revenue,Actual
4165,DOC-031700,2023-02-02,BU013,CUS063,4004,1128,Credit,1287.0,0.29,13.0,CUS063,None,Revenue,Actual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138820,DOC-031767,2025-12-09,BU013,CUS063,4004,1128,Credit,3564.0,0.29,36.0,CUS063,None,Revenue,Actual
139358,DOC-031768,2025-12-12,BU013,CUS063,4004,1128,Credit,990.0,0.29,10.0,CUS063,None,Revenue,Actual
139359,DOC-031768,2025-12-12,BU013,CUS063,4004,1128,Credit,891.0,0.29,9.0,CUS063,None,Revenue,Actual
139360,DOC-031768,2025-12-12,BU013,CUS063,4004,1128,Credit,891.0,0.29,9.0,CUS063,None,Revenue,Actual


In [63]:
import pandas as pd
from datetime import timedelta


from datetime import timedelta

def Add_terms(df_customers: pd.DataFrame, df_vendors: pd.DataFrame):
    """
    Add standard payment terms to customers and vendors.
    Reads generic_terms.csv and assigns random terms_id.
    """
    df_term = pd.read_csv("data/inputdata/generic_terms.csv")

    df_customers = df_customers.copy()
    df_vendors = df_vendors.copy()

    df_customers["terms_id"] = df_term["terms_id"].sample(
        n=len(df_customers), replace=True, random_state=42
    ).values
    df_vendors["terms_id"] = df_term["terms_id"].sample(
        n=len(df_vendors), replace=True, random_state=43
    ).values

    return df_customers, df_vendors, df_term


def generate_fact_ar_ap(df_erp, dim_customers, dim_vendors):
    """
    Create fact_ar and fact_ap tables based on ERP transactional data.
    Assumes df_erp has: document_number, date, bu_id, customer_id, vendor_id,
    amount, category, and scenario.
    """
    dim_customers, dim_vendors, dim_terms = Add_terms(dim_customers, dim_vendors)

    # Ensure proper dtypes
    df_erp = df_erp.copy()
    df_erp["date"] = pd.to_datetime(df_erp["date"])
    dim_terms["days"] = dim_terms["days"].astype(int)

    rng = np.random.default_rng(42)

    def simulate_paid_date(row):
        """Simulate paid dates with 3% overdue."""
        r = rng.random()
        if r < 0.80:
            # Paid within terms (early or on time)
            days_offset = rng.integers(int(row["days"] * 0.5), int(row["days"]))
            return row["date"] + timedelta(days=int(days_offset))
        elif r < 0.97:
            # Not paid yet
            return None
        else:
            # Overdue
            days_overdue = rng.integers(1, 15)
            return row["due_date"] + timedelta(days=int(days_overdue))

    def determine_status(row):
        """Determine payment status based on due/paid dates."""
        if pd.isna(row["paid_date"]):
            return "Not Paid"
        elif row["paid_date"] > row["due_date"]:
            return "Overdue"
        else:
            return "Paid"

    # --- A. Accounts Receivable (Revenue → Customer) ---
    df_ar = (
        df_erp[df_erp["category"].str.lower() == "revenue"]
        .groupby(["document_number", "customer_id", "bu_id"], as_index=False)
        .agg({"date": "min", "amount": "sum"})
    )

    df_ar = df_ar.merge(
        dim_customers[["customer_id", "terms_id"]],
        on="customer_id",
        how="left"
    ).merge(
        dim_terms[["terms_id", "days"]],
        on="terms_id",
        how="left"
    )

    df_ar["due_date"] = df_ar["date"] + pd.to_timedelta(df_ar["days"], unit="D")
    df_ar["paid_date"] = df_ar.apply(simulate_paid_date, axis=1)
    df_ar["status"] = df_ar.apply(determine_status, axis=1)

    df_ar = df_ar.reset_index(drop=True)
    df_ar["ar_id"] = df_ar.index + 1
    df_ar_final = df_ar[
        ["ar_id", "customer_id", "bu_id", "document_number", "date",
         "due_date", "paid_date", "amount", "terms_id", "status"]
    ].rename(columns={"document_number": "invoice_number", "date": "invoice_date"})
    df_ar_final["invoice_number"] = "INV:" + df_ar_final["invoice_number"]

    # --- B. Accounts Payable (COGS/Expense → Vendor) ---
    df_ap = (
        df_erp[df_erp["category"].str.lower().isin(["cogs", "expense", "procurement"])]
        .groupby(["document_number", "vendor_id", "bu_id"], as_index=False)
        .agg({"date": "min", "amount": "sum"})
    )

    df_ap = df_ap.merge(
        dim_vendors[["vendor_id", "terms_id"]],
        on="vendor_id",
        how="left"
    ).merge(
        dim_terms[["terms_id", "days"]],
        on="terms_id",
        how="left"
    )

    df_ap["due_date"] = df_ap["date"] + pd.to_timedelta(df_ap["days"], unit="D")
    df_ap["paid_date"] = df_ap.apply(simulate_paid_date, axis=1)
    df_ap["status"] = df_ap.apply(determine_status, axis=1)

    df_ap = df_ap.reset_index(drop=True)
    df_ap_final = df_ap[
        ["vendor_id", "bu_id", "document_number", "date",
         "due_date", "paid_date", "amount", "terms_id", "status"]
    ].rename(columns={"document_number": "invoice_number", "date": "invoice_date"})
    df_ap_final["invoice_number"] = "BILL:" + df_ap_final["invoice_number"]

    return df_ar_final, df_ap_final, dim_customers, dim_vendors, dim_terms

In [68]:
erp_data["actuals"]

,document_number,date,bu_id,party_id,account_id,item_id,debit_credit,amount,quantity,customer_id,vendor_id,scenario,amount_counter
0,700-58140500,2023-01-02,BU006,CUS030,4002,1129,Credit,187.00,187,CUS030,None,Actual,0.00
1,700-10091800,2023-01-02,BU006,CUS030,4004,1020,Credit,2580.00,20,CUS030,None,Actual,-2193.00
2,700-15315100,2023-01-02,BU013,CUS033,4001,1032,Credit,4980.00,20,CUS033,None,Actual,-4482.00
3,700-10091800,2023-01-02,BU006,CUS030,4004,1020,Credit,2580.00,20,CUS030,None,Actual,-2193.00
4,700-15315700,2023-01-02,BU013,CUS033,4001,1032,Credit,4980.00,20,CUS033,None,Actual,-4482.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066035,700-69114900,2025-12-31,BU006,CUS004,4004,1154,Credit,2697.00,3,CUS004,None,Actual,-2103.66
2066036,700-1532100,2025-12-31,BU006,CUS003,4002,1001,Credit,1749.75,5,CUS003,None,Actual,-1399.80
2066037,700-63573400,2025-12-31,BU009,CUS020,4004,1144,Credit,14995.00,5,CUS020,None,Actual,-11696.10
2066038,700-62992100,2025-12-31,BU015,CUS003,4001,1140,Credit,2995.00,5,CUS003,None,Actual,-2545.75


In [74]:

df_customers = data["customers"]
df_vendors = data["vendors"]

In [73]:
actuals = pd.read_csv("data/outputdata_v1/fact/general_ledger_actual.csv")


In [75]:
df_fact_ar, df_fact_ap, df_customers_terms, df_vendors_terms, df_terms = generate_fact_ar_ap(actuals, df_customers, df_vendors)

In [82]:
df_fact_ap

,vendor_id,bu_id,invoice_number,invoice_date,due_date,paid_date,amount,terms_id,status
0,VEN004,BU002,BILL:700-001000,2023-01-13,2023-03-14,2023-02-24,-139500.0,5,Paid
1,VEN004,BU002,BILL:700-001100,2023-01-06,2023-03-07,NaT,-139500.0,5,Not Paid
2,VEN004,BU002,BILL:700-001200,2023-01-27,2023-03-28,2023-03-18,-139500.0,5,Paid
3,VEN004,BU002,BILL:700-001300,2023-01-12,2023-03-13,2023-02-16,-139500.0,5,Paid
4,VEN004,BU002,BILL:700-001400,2023-02-24,2023-04-25,2023-04-23,-155000.0,5,Paid
...,...,...,...,...,...,...,...,...,...
71307,BU015,BU007,BILL:700-999700,2025-07-21,2025-10-19,2025-09-23,-19500.0,6,Paid
71308,BU015,BU007,BILL:700-999800,2025-08-19,2025-11-17,2025-11-04,-39000.0,6,Paid
71309,BU015,BU007,BILL:700-999800,2025-08-19,2025-11-17,2025-11-15,-39000.0,6,Paid
71310,BU015,BU007,BILL:700-999900,2025-09-16,2025-12-15,2025-11-23,-39000.0,6,Paid


In [83]:
df_fact_ar.to_csv("data/cbs_new_data/fact/fact_ar.csv", index=False)
df_fact_ap.to_csv("data/cbs_new_data/fact/fact_ap.csv", index=False)
df_customers_terms.to_csv("data/cbs_new_data/dim/customers.csv", index=False)
df_vendors_terms.to_csv("data/cbs_new_data/dim/vendors.csv", index=False)
df_terms.to_csv("data/cbs_new_data/dim/terms.csv", index=False)

In [ ]:
df_in

In [42]:
df_fact_ar[df_fact_ar["status"] == "Overdue"]

,ar_id,customer_id,bu_id,invoice_number,invoice_date,due_date,paid_date,amount,terms_id,status
15,16,CUS008,BU005,INV:700-002500,2023-01-04,2023-01-19,2023-01-24,4549.35,7,Overdue
124,125,CUS008,BU005,INV:700-013400,2023-01-11,2023-01-26,2023-01-29,4899.30,7,Overdue
141,142,CUS008,BU005,INV:700-015100,2023-01-13,2023-01-28,2023-02-08,4899.30,7,Overdue
208,209,CUS008,BU005,INV:700-021800,2023-01-12,2023-01-27,2023-02-08,4899.30,7,Overdue
272,273,CUS008,BU005,INV:700-028200,2023-02-08,2023-02-23,2023-02-24,4899.30,7,Overdue
...,...,...,...,...,...,...,...,...,...,...
286667,286668,CUS079,BU005,INV:700-9993700,2025-01-22,2025-04-22,2025-05-06,4465.00,6,Overdue
286689,286690,CUS079,BU005,INV:700-9995700,2025-04-04,2025-07-03,2025-07-14,4578.00,6,Overdue
286691,286692,CUS079,BU005,INV:700-9995900,2025-05-06,2025-08-04,2025-08-16,4565.00,6,Overdue
286698,286699,CUS079,BU005,INV:700-9996500,2025-06-19,2025-09-17,2025-09-26,4542.00,6,Overdue


In [44]:
df_terms

,terms_id,terms,days,description
0,1,Net 7,7,Payment due within 7 days of invoice date.
1,2,Net 14,14,Payment due within 14 days of invoice date.
2,3,Net 30,30,Standard business payment term; due in 30 days.
3,4,Net 45,45,Extended term for preferred partners.
4,5,Net 60,60,Payment due within 60 days; large corporate ac...
5,6,Net 90,90,Long-term payment agreements for strategic cus...
6,7,15 Days EOM,15,Payment due 15 days after the end of the invoi...
7,8,30 Days EOM,30,Payment due 30 days after the end of the invoi...


In [84]:
df_inventory = pd.read_csv("data/outputdata_v1/fact/inventory_lines.csv")

In [105]:
def star_extra_cat(df_extra_cat: pd.DataFrame, prefix: str, id_start: int):
    # 1) Map product names in df_revenue to product_id

    def generate_item_id(prefix="ITEM", start_num=1000):
        """Generate sequential item IDs with a prefix"""
        counter = start_num
        while True:
            yield f"{prefix}{counter:04d}"
            counter += 1
    df_extra_cat = df_extra_cat[df_extra_cat["annual_spend"] != 0]
    df_extra_cat = df_extra_cat.dropna(subset=["party_id"])
    
    df_extra_parties = df_extra_cat[["party_id", "party_name"]].drop_duplicates("party_name").dropna()
    df_extra_items = df_extra_cat[["item_name", "unit_price"]].drop_duplicates("item_name")
    item_id_gen_extra = generate_item_id("", id_start)
    df_extra_items["item_id"] = [next(item_id_gen_extra) for _ in range(len(df_extra_items))]
    
    rev_map = df_extra_items[["item_id","item_name"]]
    df_extra_cat = df_extra_cat.merge(rev_map, left_on="item_name", right_on="item_name", how="left")
    df_extra_cat["item_id"] = df_extra_cat["item_id"].astype("Int64")
    
    df_extra_cat["quantity"] = np.ceil(df_extra_cat["annual_spend"] / df_extra_cat["unit_price"]).astype(int)
    df_extra_cat = df_extra_cat.drop(columns=["item_name","item_name", "bu_name","party_name", "markdown"], inplace=False)
    df_extra_cat.rename(columns={"annual_spend": "amount"}, inplace=True)

    df_extra_cat = df_extra_cat[["bu_id", "party_id","account_id", "item_id", "quantity", "unit_price", "amount", "category"]]
    return df_extra_cat, df_extra_parties, df_extra_items

In [103]:
df_inventory

,bu_id,bu_name,party_id,party_name,account_id,item_name,unit_price,markdown,category,annual_spend
0,BU002,Lego Billund Factory,VEN003,INEOS Styrolution,1003,ABS plastic resin pellets,27500.0,0.12,Inventory,4238800.0
1,BU002,Lego Billund Factory,VEN004,Covestro,1003,Polycarbonate granules,29300.0,0.13,Inventory,3244500.0
2,BU002,Lego Billund Factory,VEN009,Clariant,1003,Color masterbatch additives,18500.0,0.15,Inventory,1412900.0
3,BU005,Lego Kladno Factory,VEN005,Braskem,1003,Bio-based polyethylene,31200.0,0.11,Inventory,2564200.0
4,BU005,Lego Kladno Factory,VEN073,Toray Industries,1003,Engineering plastics,22600.0,0.13,Inventory,1308300.0
...,...,...,...,...,...,...,...,...,...,...
164,BU005,Lego Kladno Factory,BU008,Lego Monterrey Factory,1006,Intercompany inventory transfer - Kladno to Mo...,32000.0,0.00,Inventory,100.0
165,BU008,Lego Monterrey Factory,BU005,Lego Kladno Factory,1006,Intercompany inventory transfer - Monterrey to...,27000.0,0.00,Inventory,0.0
166,BU002,Lego Billund Factory,BU001,Lego Billund Factory,1006,Intercompany inventory adjustment - Billund,15000.0,0.00,Inventory,0.0
167,BU005,Lego Kladno Factory,BU001,Lego Billund Factory,1006,Intercompany inventory adjustment - Kladno,11000.0,0.00,Inventory,0.0


In [106]:
df_inventory_clean, df_inv_parties, df_inv_items = star_extra_cat(df_inventory, "INV", 4000)

In [110]:
df_inventory_clean.to_csv("data/cbs_new_data/fact/fact_inventory.csv", index=False)
df_inv_parties.to_csv("data/cbs_new_data/dim/vendors_inventory.csv", index=False)
df_inv_items.to_csv("data/cbs_new_data/dim/items_inventory.csv", index=False)

In [ ]:
df_revenue["annual_cost"] = df_revenue["annual_spend"]*(1-df_revenue["markdown"])


,bu_id,party_id,account_id,unit_price,markdown,category,annual_spend,item_id,annual_cost
0,BU006,CUS003,4004,349.95,0.14,Revenue,18414200.0,1000,15836212.0
1,BU006,CUS003,4002,349.95,0.20,Revenue,7128100.0,1001,5702480.0
2,BU006,CUS003,4001,399.95,0.10,Revenue,15444200.0,1002,13899780.0
3,BU006,CUS004,4004,749.95,0.13,Revenue,16632200.0,1003,14470014.0
4,BU006,CUS004,4002,749.95,0.18,Revenue,5940100.0,1004,4870882.0
...,...,...,...,...,...,...,...,...,...
223,BU015,BU007,4007,19500.00,0.19,Revenue,352100.0,1173,285201.0
224,BU003,BU002,4007,47000.00,0.20,Revenue,1631500.0,1174,1305200.0
225,BU001,BU005,4007,41000.00,0.21,Revenue,1087700.0,1175,859283.0
226,BU001,BU008,4007,38500.00,0.22,Revenue,699200.0,1176,545376.0


In [25]:
actuals

NameError: name 'actuals' is not defined